In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/27 20:27:21 WARN Utils: Your hostname, Desktop-Gar resolves to a loopback address: 127.0.1.1; using 172.25.243.204 instead (on interface eth0)
24/02/27 20:27:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 20:27:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## How many taxi trips were there on the 15th of October?

In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [4]:
# create 24 partitions in our dataframe
df = df.repartition(6)
# parquetize and write to fhvhv/2021/01/ folder
df.write.parquet('fhvhv/2019/10/', mode='overwrite')

In [5]:
df = spark.read.parquet('fhvhv/2019/10/*')

In [6]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01437|2019-10-18 16:58:08|2019-10-18 17:11:20|         264|         130|   null|                B01437|
|              B03048|2019-10-11 09:20:25|2019-10-11 09:37:15|         264|         264|   null|                B03048|
|              B01328|2019-10-08 09:02:00|2019-10-08 09:16:00|          78|         242|   null|                B02534|
|              B00310|2019-10-17 14:52:53|2019-10-17 15:07:03|         264|          60|   null|                B00310|
|              B01239|2019-10-04 12:15:42|2019-10-04 12:42:54|         264|         247|   null|                B03127|
|              B01300|2019-10-30 12:06:2

In [7]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [8]:
df[df['pickup_datetime'].between('2019-10-15 00:00:00', '2019-10-15 59:59:59')].count()


62610

## What is the length of the longest trip in the dataset in hours?

In [9]:
from pyspark.sql.functions import *

In [10]:
df = df.withColumn('Result', to_timestamp('dropOff_datetime') - to_timestamp('pickup_datetime'))

In [11]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|              Result|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B01437|2019-10-18 16:58:08|2019-10-18 17:11:20|         264|         130|   null|                B01437|INTERVAL '0 00:13...|
|              B03048|2019-10-11 09:20:25|2019-10-11 09:37:15|         264|         264|   null|                B03048|INTERVAL '0 00:16...|
|              B01328|2019-10-08 09:02:00|2019-10-08 09:16:00|          78|         242|   null|                B02534|INTERVAL '0 00:14...|
|              B00310|2019-10-17 14:52:53|2019-10-17 15:07:03|         264|          60|   null|                B00310|INTERVAL '0 00:14...|
|            

In [12]:
df.createOrReplaceTempView('trips_data')

In [13]:
spark.sql("""
SELECT
    (max(Result)/60/60)
FROM
    trips_data
""").show()

+-------------------------+
|((max(Result) / 60) / 60)|
+-------------------------+
|     INTERVAL '7 07:19...|
+-------------------------+



## Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

In [14]:
dfzone = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [15]:
dfzone.write.mode("overwrite").parquet('zones')

In [16]:
!ls -lh zones

total 8.0K
-rwxrwxrwx 1 root root    0 Feb 27 20:28 _SUCCESS
-rwxrwxrwx 1 root root 5.8K Feb 27 20:28 part-00000-cddfb760-302c-408b-8388-572929db314a-c000.snappy.parquet


In [17]:
dfzone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|              Result|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B01437|2019-10-18 16:58:08|2019-10-18 17:11:20|         264|         130|   null|                B01437|INTERVAL '0 00:13...|
|              B03048|2019-10-11 09:20:25|2019-10-11 09:37:15|         264|         264|   null|                B03048|INTERVAL '0 00:16...|
|              B01328|2019-10-08 09:02:00|2019-10-08 09:16:00|          78|         242|   null|                B02534|INTERVAL '0 00:14...|
|              B00310|2019-10-17 14:52:53|2019-10-17 15:07:03|         264|          60|   null|                B00310|INTERVAL '0 00:14...|
|            

In [19]:
dfzone.createOrReplaceTempView('zone_data')

In [20]:
spark.sql("""
WITH least as (
    SELECT PUlocationID, count(*)
    FROM trips_data
    GROUP BY PUlocationID
    ORDER BY count(*) LIMIT 1
)
SELECT zone 
FROM zone_data
JOIN least ON PUlocationID = LocationID
""").show()

+-----------+
|       zone|
+-----------+
|Jamaica Bay|
+-----------+

